In [1]:
# Import modules
import pandas as pd
import numpy as np
import re
from functools import reduce

# Set pandas number format
pd.options.display.float_format = "{:,.2f}".format

# Load and clean data


In [2]:
dir_path = "../../data/exiobase/_ixi/IOT_2014_ixi/"
rexia_dir_path = "../../data/exiobase/REX/rexia-data/converted/"

In [3]:
# Import A, Y and x from Exiobase 3.8
# ! 'A' is a large datasets, mind load time
A = pd.read_csv(f"{dir_path}A.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])
# Z = pd.read_csv(f"{dir_path}Z.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])
Y = pd.read_csv(f"{dir_path}Y.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])
x = pd.read_csv(f"{dir_path}x.txt", delimiter="\t", header=0, index_col=[0, 1])["indout"]
# Import F, F_y (extensions) from both satellite and impacts
F_satellite = pd.read_csv(
    f"{dir_path}satellite/F.txt", delimiter="\t", header=[0, 1], index_col=[0]
)
F_y_satellite = pd.read_csv(
    f"{dir_path}satellite/F_Y.txt", delimiter="\t", header=[0, 1], index_col=[0]
)
F_unit_satellite = pd.read_csv(
    f"{dir_path}satellite/unit.txt", delimiter="\t", header=0, index_col=[0]
)
F_impact = pd.read_csv(f"{dir_path}impacts/F.txt", delimiter="\t", header=[0, 1], index_col=[0])
F_y_impact = pd.read_csv(f"{dir_path}impacts/F_Y.txt", delimiter="\t", header=[0, 1], index_col=[0])
F_unit_impact = pd.read_csv(f"{dir_path}impacts/unit.txt", delimiter="\t", header=0, index_col=[0])
# ! Also import Q (extension) from REXIA (Cabernard & Pfister, 2022)
Q = pd.read_csv(f"{rexia_dir_path}Q_REXIA.csv", header=[0, 1], index_col=[0])

In [4]:
# get multiindex
region_and_sector_labels = A.index

# get each level using  'get_level_values()'
region_labels = A.index.get_level_values(level=0).unique()
sector_labels = A.index.get_level_values(level=1).unique()
FD_sector_labels = Y.columns.get_level_values(level=1).unique()

In [5]:
# Some sector labels of Q do not match those of Exiobase
wrong_labels = [
    '"Fishing, operating of fish hatcheries and fish farms; service activities incidental to fishing (05)"',
    '"Manufacture of furniture; manufacturing n.e.c. (36)"',
    '"Manufacture of gas; distribution of gaseous fuels through mains"',
    '"Manufacture of wearing apparel; dressing and dyeing of fur (18)"',
    '"Manufacture of wood and of products of wood and cork, except furniture; manufacture of articles of straw and plaiting materials (20)"',
    '"Mining of coal and lignite; extraction of peat (10)"',
    '"Public administration and defence; compulsory social security (75)"',
    '"Retail trade, except of motor vehicles and motorcycles; repair of personal and household goods (52)"',
    '"Supporting and auxiliary transport activities; activities of travel agencies (63)"',
    '"Tanning and dressing of leather; manufacture of luggage, handbags, saddlery, harness and footwear (19)"',
]
correct_labels = [
    "Fishing, operating of fish hatcheries and fish farms; service activities incidental to fishing (05)",
    "Manufacture of furniture; manufacturing n.e.c. (36)",
    "Manufacture of gas; distribution of gaseous fuels through mains",
    "Manufacture of wearing apparel; dressing and dyeing of fur (18)",
    "Manufacture of wood and of products of wood and cork, except furniture; manufacture of articles of straw and plaiting materials (20)",
    "Mining of coal and lignite; extraction of peat (10)",
    "Public administration and defence; compulsory social security (75)",
    "Retail trade, except of motor vehicles and motorcycles; repair of personal and household goods (52)",
    "Supporting and auxiliary transport activities; activities of travel agencies (63)",
    "Tanning and dressing of leather; manufacture of luggage, handbags, saddlery, harness and footwear (19)",
]

sectors_mapping = {wrong_labels[i]: correct_labels[i] for i, _ in enumerate(wrong_labels)}
Q.rename(columns=sectors_mapping, inplace=True)

# make sure that both indexes are the same
sorted(sector_labels) == sorted(Q.columns.get_level_values(level=1).unique())

True

In [6]:
# Aggregate countries so as to match the region clusters of Exiobase

# Use the mapping file to determine which countries is associated to which cluster
# And also change to country code instead of the full country name
countries_mapping = pd.read_csv("./countries_mapping.csv", header=None)
countries_mapping.set_index(0, inplace=True)

# check that each country is associated with a region associated in the csv file
for country in list(Q.columns.get_level_values(0).unique()):
    if len(countries_mapping.loc[country, 1]) != 2:
        print(country)

# Assign each country to its region
clusters = dict()
for code in countries_mapping[1]:
    clusters[code] = []
for country in list(Q.columns.get_level_values(0).unique()):
    clusters[countries_mapping.loc[country, 1]].append(country)

# check that we have the right number of clusters
print((len(clusters) == 49) & (sorted(region_labels) == sorted(clusters.keys())))

True


In [7]:
# Update Q with the new region and sector labels
new_Q = []
for code, countries in clusters.items():
    res = Q.loc[:, clusters[code]].groupby(level=1, axis=1).sum().loc[:, sector_labels]
    res = pd.concat([res], keys=[code], axis=1)
    new_Q.append(res)

new_Q = pd.concat(new_Q, axis=1)
new_Q.columns.set_names(["region", "sector"], inplace=True)
new_Q.index.set_names(["stressor"], inplace=True)
del Q
new_Q

region                                                                    AT   
sector                                             Cultivation of paddy rice   
stressor                                                                       
BGS mining data [t]                                                     0.00  \
Land use area of mining ? primary commodity all...                      0.00   
Land-use related biodiversity loss of mining ? ...                      0.00   
Land use area of mining ? monetary allocation [...                      0.00   
Land-use related biodiversity loss of mining ? ...                      0.00   
Total material footprint [kt]                                           0.00   
Climate change impacts [kg CO2-eq]                                      0.00   
Particulate-matter related health impacts [DALYs]                       0.00   
Blue water consumption [Mio. m3]                                        0.00   
Water stress [Mio. m3 H2O-eq]                                           0.00   
Total land use area based on EXIOBASE3 [km2]                            0.00   
Total land-use related biodiversity loss based ...                      0.00   
Total land use area based on FAOSTAT for agricu...                      0.00   
Total land-use related biodiversity loss based ...                      0.00   

region                                                                    
sector                                             Cultivation of wheat   
stressor                                                                  
BGS mining data [t]                                                0.00  \
Land use area of mining ? primary commodity all...                 0.00   
Land-use related biodiversity loss of mining ? ...                 0.00   
Land use area of mining ? monetary allocation [...                 0.00   
Land-use related biodiversity loss of mining ? ...                 0.00   
Total material footprint [kt]                                  1,632.32   
Climate change impacts [kg CO2-eq]                       564,031,974.79   
Particulate-matter related health impacts [DALYs]                237.72   
Blue water consumption [Mio. m3]                                   0.79   
Water stress [Mio. m3 H2O-eq]                                      1.00   
Total land use area based on EXIOBASE3 [km2]                   3,118.04   
Total land-use related biodiversity loss based ...         5,138,529.87   
Total land use area based on FAOSTAT for agricu...           304,645.00   
Total land-use related biodiversity loss based ...         5,250,904.40   

region                                                                                
sector                                             Cultivation of cereal grains nec   
stressor                                                                              
BGS mining data [t]                                                            0.00  \
Land use area of mining ? primary commodity all...                             0.00   
Land-use related biodiversity loss of mining ? ...                             0.00   
Land use area of mining ? monetary allocation [...                             0.00   
Land-use related biodiversity loss of mining ? ...                             0.00   
Total material footprint [kt]                                              3,133.72   
Climate change impacts [kg CO2-eq]                                   818,868,108.21   
Particulate-matter related health impacts [DALYs]                            364.99   
Blue water consumption [Mio. m3]                                               6.86   
Water stress [Mio. m3 H2O-eq]                                                  8.68   
Total land use area based on EXIOBASE3 [km2]                               4,459.47   
Total land-use related biodiversity loss based ...                     8,824,797.53   
Total land use area based on FAOSTAT for agricu...        

# Compute Leontief


In [8]:
# Create an identity matrix the same order
Id = np.identity(len(region_and_sector_labels))

# Compute leontief inverse matrix in the Demand-pull model
L_values = np.linalg.inv((Id - A))
L = pd.DataFrame(data=L_values, index=region_and_sector_labels, columns=region_and_sector_labels)

# compute extension intensity
f_satellite = F_satellite / x.T
f_satellite = f_satellite.replace([np.inf, -np.inf, np.nan], 0)  # ! don't forget to replace

f_impact = F_impact / x.T
f_impact = f_impact.replace([np.inf, -np.inf, np.nan], 0)  # ! don't forget to replace

# compute extension intensity
f_rexia = new_Q / x.T
f_rexia = f_rexia.replace([np.inf, -np.inf, np.nan], 0)  # ! don't forget to replace

# 3) Supply chain of metals


In [9]:
# Derive index
# ! (matlab code lines 767-796)

METALS = sector_labels[np.r_[24:31, 70:83]]

index_t = pd.IndexSlice[:, METALS]
index_t = A.loc[index_t, :].index  # to have explicit labels instead of slice(None)

index_o = region_and_sector_labels.difference(index_t)

# Show (first) results
index_t[:5]

MultiIndex([('AT', 'Mining of iron ores'),
            ('BE', 'Mining of iron ores'),
            ('BG', 'Mining of iron ores'),
            ('CY', 'Mining of iron ores'),
            ('CZ', 'Mining of iron ores')],
           names=['region', 'sector'])

In [10]:
# Derive the Leontief inverse for the remaining economy
# ! (matlab code lines 798 - 801)

# Create an identity matrix the same order
Id = np.identity(len(index_o))

# get corresponding (numerical) index for index_o
ix_o = np.sort(region_and_sector_labels.get_indexer_for(index_o))

# Compute leontief inverse matrix in the Demand-pull model (for the remaining economy)
L_oo_dash = pd.DataFrame(
    data=np.zeros(shape=A.shape),
    index=region_and_sector_labels,
    columns=region_and_sector_labels,
)
L_oo_dash.iloc[ix_o, ix_o] = np.linalg.inv(Id - A.iloc[ix_o, ix_o])

# save intermediary calculation that will be reused multiple times
Y_region = Y.groupby(by="region", axis=1).sum()
Y_region = Y_region.loc[region_and_sector_labels, region_labels]  # reorder rows / cols
Y_global = Y_region.sum(axis=1)
AtoLoo = A.loc[index_t, index_o] @ L_oo_dash.loc[index_o, index_o]

In [11]:
# Show results
Y_region

region                                                        AT   BE   BG   
region sector                                                                
AT     Cultivation of paddy rice                            0.00 0.00 0.00  \
       Cultivation of wheat                                 0.00 0.10 2.20   
       Cultivation of cereal grains nec                   152.07 0.32 0.37   
       Cultivation of vegetables, fruit, nuts             803.64 0.33 1.37   
       Cultivation of oil seeds                             0.00 0.14 3.48   
...                                                          ...  ...  ...   
WM     Activities of membership organisation n.e.c. (91)    1.06 3.42 2.26   
       Recreational, cultural and sporting activities ...   9.15 2.20 3.83   
       Other service activities (93)                        1.54 3.59 0.14   
       Private households with employed persons (95)        0.17 1.36 0.02   
       Extra-territorial organizations and bodies           0.00 0.00 0.00   

region                                                      CY   CZ    DE   
region sector                                                               
AT     Cultivation of paddy rice                          0.00 0.00  0.00  \
       Cultivation of wheat                               0.01 0.21 14.69   
       Cultivation of cereal grains nec                   0.02 0.50 13.44   
       Cultivation of vegetables, fruit, nuts             0.09 6.68 98.67   
       Cultivation of oil seeds                           0.00 0.52 24.16   
...                                                        ...  ...   ...   
WM     Activities of membership organisation n.e.c. (91)  0.23 0.51  5.08   
       Recreational, cultural and sporting activities ... 3.69 8.63 13.57   
       Other service activities (93)                      0.01 0.39  2.65   
       Private households with employed persons (95)      0.16 0.10  1.41   
       Extra-territorial organizations and bodies         0.00 0.00  0.00   

region                                                       DK   EE    ES   
region sector                                                                
AT     Cultivation of paddy rice                           0.00 0.00  0.00  \
       Cultivation of wheat                                0.00 0.01  0.28   
       Cultivation of cereal grains nec                    0.01 0.03  0.15   
       Cultivation of vegetables, fruit, nuts              0.60 0.03  4.29   
       Cultivation of oil seeds                            0.83 0.00  0.03   
...                                                         ...  ...   ...   
WM     Activities of membership organisation n.e.c. (91)   2.78 0.37  4.85   
       Recreational, cultural and sporting activities ... 14.07 0.29 20.33   
       Other service activities (93)                       1.06 0.02  5.36   
       Private households with employed persons (95)       0.07 0.02 90.87   
       Extra-territorial organizations and bodies          0.00 0.00  0.00   

region                                                       FI  ...   TR   
region sector                                                    ...        
AT     Cultivation of paddy rice                           0.00  ... 0.00  \
       Cultivation of wheat                                0.01  ... 0.01   
       Cultivation of cereal grains nec                    0.03  ... 0.03   
       Cultivation of vegetables, fruit, nuts              1.28  ... 0.48   
       Cultivation of oil seeds                            0.16  ... 0.00   
...                                                         ...  ...  ...   
WM     Activities of membership organisation n.e.c. (91)   1.29  ... 1.09   
       Recreational, cultural and sporting activities ... 10.59  ... 0.12   
       Other service activities (93)                       1.64  ... 3.62   
       Private households with employed persons (95)       0.10  ... 1.00   
       Extra-territorial organizations and b

In [12]:
# Show results
Y_global

region  sector                                             
AT      Cultivation of paddy rice                                  0.00
        Cultivation of wheat                                     112.16
        Cultivation of cereal grains nec                         192.44
        Cultivation of vegetables, fruit, nuts                 1,033.94
        Cultivation of oil seeds                                  41.69
                                                                 ...   
WM      Activities of membership organisation n.e.c. (91)      9,206.81
        Recreational, cultural and sporting activities (92)   29,868.39
        Other service activities (93)                         18,556.71
        Private households with employed persons (95)          4,022.69
        Extra-territorial organizations and bodies                 0.00
Length: 7987, dtype: float64

## Choice of stressor


In [13]:
# Select stressors to be analysed
# ! (matlab code line 803)

# # Option 1: if we precisely know which stressor we want to analyse
# idx = [
#     "Domestic Extraction Used - Metal Ores - Nickel ores",
#     "Unused Domestic Extraction - Metal Ores - Nickel ores",
# ]

# # Option 2: find all the stressors that contain the word...
# stressor = "Water Withdrawal Blue - Manufacturing"
# MASK = d.index.str.contains(stressor, flags=re.IGNORECASE, regex=True)
# idx = d.loc[MASK].index
# print(*list(zip(d.index.get_indexer_for(idx), idx)), sep="\n")

# Option 3: use Q from REXIA (Cabernard & Pfister, 2022)
d = f_rexia  # choose f_impact / f_satellite / f_biodiv
stressor_labels = d.index  # .get_level_values(level=1).unique()
list(enumerate(d.index))

[(0, 'BGS mining data [t]'),
 (1, 'Land use area of mining ? primary commodity allocation [km2]'),
 (2,
  'Land-use related biodiversity loss of mining ? primary commodity allocation [global pdf]'),
 (3, 'Land use area of mining ? monetary allocation [km2]'),
 (4,
  'Land-use related biodiversity loss of mining ? monetary allocation [global pdf]'),
 (5, 'Total material footprint [kt]'),
 (6, 'Climate change impacts [kg CO2-eq]'),
 (7, 'Particulate-matter related health impacts [DALYs]'),
 (8, 'Blue water consumption [Mio. m3]'),
 (9, 'Water stress [Mio. m3 H2O-eq]'),
 (10, 'Total land use area based on EXIOBASE3 [km2]'),
 (11,
  'Total land-use related biodiversity loss based on EXIOBASE3 [10^12 global pdf]'),
 (12,
  'Total land use area based on FAOSTAT for agriculture and forestry [km2]'),
 (13,
  'Total land-use related biodiversity loss based on FAOSTAT for agriculture and forestry [10^12 global pdf]')]

In [50]:
d_i = d.iloc[np.r_[4]].sum()
stress = "biodiv"  # for the sankey (excel) data

# save intermediary calculation that will be reused multiple times
dLt = d_i @ L.loc[:, index_t]

# # ! make sure it is a column vector (= pd.Series). use sum() if necessary
d_i

region  sector                                             
AT      Cultivation of paddy rice                             0.00
        Cultivation of wheat                                  0.00
        Cultivation of cereal grains nec                      0.00
        Cultivation of vegetables, fruit, nuts                0.00
        Cultivation of oil seeds                              0.00
                                                              ... 
WM      Activities of membership organisation n.e.c. (91)     0.00
        Recreational, cultural and sporting activities (92)   0.00
        Other service activities (93)                         0.00
        Private households with employed persons (95)         0.00
        Extra-territorial organizations and bodies            0.00
Name: Land-use related biodiversity loss of mining ? monetary allocation [global pdf], Length: 7987, dtype: float64

## Link Preg Treg

$$E_{P,T} = diag\biggl(d_{i}\biggr) \times L_{all, T} \times diag\biggl(Y_{all, T} + A_{T,O} \times L'_{O,O} \times Y_{all, O}\biggr)$$


Important notes:

- `*` in **Matlab** = `@` in **Python** (Pandas / Numpy) = **dot product**
- `*` in **Python** = **element-wise multiplication**
- `A * diag(B)` in Matlab = `A @ diag(B)` = `A * B.T`
- `diag(A) * B` in Matlab = `diag(A) @ B` = `( A * B.T ).T`


In [51]:
# ! (matlab code lines 805-812)

E_P_T = pd.DataFrame(
    data=np.zeros(shape=A.shape),
    index=region_and_sector_labels,
    columns=region_and_sector_labels,
)

# Compute Lall-t . diag( Yall-t +  Ato.Loo.Yall-o)
Lt_x_Yt_plus_AtoLooY_GLO = (
    L.loc[:, index_t] * (Y_global.loc[index_t] + AtoLoo @ Y_global.loc[index_o]).T
)

# Compute diag(di) . Lall-t . diag( Yall-t +  Ato.Loo.Yall-o)
E_P_T.loc[:, index_t] = (d_i * Lt_x_Yt_plus_AtoLooY_GLO.T).T.values

# sum the sectors together and keep regions disaggregated along both axis
E_Preg_Treg = E_P_T.groupby("region", axis=0).sum().groupby("region", axis=1).sum()

# reorder rows and cols
E_Preg_Treg = E_Preg_Treg.loc[region_labels, region_labels]

# delete variables that are no longer used to save memory
del Lt_x_Yt_plus_AtoLooY_GLO

# show results
E_Preg_Treg

region,AT,BE,BG,CY,CZ,DE,DK,EE,ES,FI,...,TR,TW,NO,ID,ZA,WA,WL,WE,WF,WM
region,,,,,,,,,,,,,,,,,,,,,
AT,"19,254.09",18.33,1.16,0.19,6.83,68.29,0.77,0.13,21.44,1.95,...,64.47,52.03,5.06,6.57,3.36,229.86,21.36,3.58,39.92,"5,428.63"
BE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
BG,339.42,"4,984.00","62,568.60",3.47,203.75,"15,681.05",19.00,24.06,223.56,"6,102.12",...,"3,173.27",604.89,180.36,26.18,70.76,"1,549.28",42.61,265.56,283.80,712.64
CY,5.77,6.62,1.79,"29,692.79",2.45,16.33,3.38,0.35,17.64,5.55,...,10.62,6.86,1.84,54.50,2.50,12.16,10.47,4.29,4.40,10.75
CZ,336.66,50.31,10.09,0.12,"23,685.38","1,641.70",17.10,2.45,73.82,24.25,...,79.24,16.57,37.06,2.38,2.20,45.61,21.70,48.63,22.70,63.11
DE,241.66,388.11,8.22,1.01,470.50,"24,858.49",31.84,3.64,784.82,55.08,...,196.89,80.64,67.32,9.46,21.00,225.66,67.59,38.58,83.22,"2,578.04"
DK,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
EE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ES,367.96,"7,169.72","52,240.92",6.42,172.29,"4,245.87",14.97,7.10,"51,788.11",83.97,...,"5,412.91","10,955.07",110.32,61.49,16.13,"1,718.14",186.66,461.97,589.46,"1,230.71"


## Link Treg FDreg

$$E_{T, FDreg} = diag\biggl(d_{i} \times L_{all, T}\biggr) \times \biggl(Y_{T, reg} + A_{T,O} \times L'_{O,O} \times Y_{O, reg}\biggr)$$


Important notes:

- `*` in **Matlab** = `@` in **Python** (Pandas / Numpy) = **dot product**
- `*` in **Python** = **element-wise multiplication**
- `A * diag(B)` in Matlab = `A @ diag(B)` = `A * B.T`
- `diag(A) * B` in Matlab = `diag(A) @ B` = `( A * B.T ).T`


In [52]:
# ! (matlab code lines 814-820)

E_T_FDreg = pd.DataFrame(
    data=np.zeros(shape=(len(region_and_sector_labels), len(region_labels))),
    index=region_and_sector_labels,
    columns=region_labels,
)

# Compute Yrt + Ato.Loo.Yro
Yt_plus_AtoLooY_REG = Y_region.loc[index_t, :] + AtoLoo @ Y_region.loc[index_o, :]

# Compute diag(di) .  Lall-t  .  ( Yrt + Ato.Loo.Yro )
E_T_FDreg.loc[index_t, :] = (dLt * Yt_plus_AtoLooY_REG.T).T.values

# group sectors together (along the rows) and keep regions disaggregated
E_Treg_FDreg = E_T_FDreg.groupby("region", axis=0).sum().loc[region_labels]

# reorder rows and cols
E_Treg_FDreg = E_Treg_FDreg.loc[region_labels, region_labels]

# delete variables that are no longer used to save memory space
del E_T_FDreg

# Show results
E_Treg_FDreg

region,AT,BE,BG,CY,CZ,DE,DK,EE,ES,FI,...,TR,TW,NO,ID,ZA,WA,WL,WE,WF,WM
region,,,,,,,,,,,,,,,,,,,,,
AT,"27,543.45",541.87,180.02,16.51,685.88,"10,045.21",256.68,49.30,845.08,221.73,...,891.30,243.77,414.85,196.08,120.09,"2,814.66","1,001.70",479.96,899.37,"12,031.21"
BE,"3,884.27","74,784.12",405.43,88.17,"1,729.29","43,722.60","1,290.74",229.66,"7,199.10","1,385.13",...,"6,268.85",980.82,"2,047.02","1,262.27","1,051.67","22,227.59","7,293.27","1,505.31","11,293.74","41,766.68"
BG,"1,618.95","5,359.28","46,025.08",89.68,998.78,"14,194.92",566.80,120.68,"2,079.86",781.68,...,"30,942.11","10,830.42",773.45,821.37,"8,303.28","13,459.66","2,796.15","2,143.14","4,615.27","22,055.04"
CY,257.17,244.42,82.35,"15,893.59",207.13,438.82,184.81,112.25,312.49,231.18,...,443.83,321.47,335.74,192.64,194.02,460.29,435.97,276.07,422.33,"1,169.18"
CZ,"1,727.61",791.66,235.06,14.42,"18,252.07","12,565.32",327.75,69.19,"1,072.75",307.58,...,"1,213.19",244.33,596.40,144.23,93.82,"2,075.83","1,074.10",598.58,"1,019.42","2,530.41"
DE,"17,141.76","13,360.22","1,764.56",178.75,"8,851.45","235,633.21","5,272.09",601.13,"15,279.62","3,722.53",...,"17,920.49","4,617.67","6,643.79","3,199.84","2,213.87","50,327.92","17,429.90","4,394.82","16,068.92","53,056.82"
DK,65.93,98.03,11.28,2.01,61.40,965.97,"3,583.01",21.91,149.95,132.01,...,232.29,33.98,567.32,30.71,24.37,375.73,210.22,64.67,171.98,340.17
EE,9.22,37.10,26.54,1.53,7.36,116.79,24.91,192.81,14.56,136.53,...,147.45,8.47,95.42,6.16,3.02,113.41,28.94,25.97,29.03,88.45
ES,"2,195.27","4,303.62",519.74,164.70,"1,417.64","31,166.79","1,002.88",164.03,"141,747.87","1,032.63",...,"27,243.40","14,755.22","2,126.26","1,547.44",898.96,"18,800.93","16,598.37","1,995.45","31,416.91","46,114.24"


## Link Preg FDreg

$$E_{P, FDreg} = diag(d_{i}) \times L_{all, T} \times (Y_{T, reg} + A_{T,O} \times L'_{O,O} \times Y_{O, reg})$$


Important notes:

- `*` in **Matlab** = `@` in **Python** (Pandas / Numpy) = **dot product**
- `*` in **Python** = **element-wise multiplication**
- `A * diag(B)` in Matlab = `A @ diag(B)` = `A * B.T`
- `diag(A) * B` in Matlab = `diag(A) @ B` = `( A * B.T ).T`


In [53]:
# ! (matlab code lines 822-827)

# Compute  Lgt . ( Yrt + Ato.Loo.Yro )
Lt_dot_Yt_plus_AtoLooY_REG = L.loc[:, index_t] @ Yt_plus_AtoLooY_REG

# Compute  diag(di) .  Lall-t . ( Yrt + Ato.Loo.Yro )
E_P_FDreg = (d_i * Lt_dot_Yt_plus_AtoLooY_REG.T).T

# group sectors together (along the rows) and keep regions disaggregated
E_Preg_FDreg = E_P_FDreg.groupby("region", axis=0).sum()

# reorder rows and cols
E_Preg_FDreg = E_Preg_FDreg.loc[region_labels, region_labels]

# delete variables that are no longer used to save memory space
del Yt_plus_AtoLooY_REG, Lt_dot_Yt_plus_AtoLooY_REG, E_P_FDreg

# show results
E_Preg_FDreg

region,AT,BE,BG,CY,CZ,DE,DK,EE,ES,FI,...,TR,TW,NO,ID,ZA,WA,WL,WE,WF,WM
region,,,,,,,,,,,,,,,,,,,,,
AT,"6,028.75",100.42,16.98,7.09,56.28,646.46,31.03,4.04,166.46,28.22,...,157.95,50.55,48.44,124.89,47.65,841.80,262.05,56.54,485.88,"12,221.39"
BE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
BG,717.73,"2,487.22","10,000.09",29.92,442.70,"7,866.62",289.88,73.24,"1,097.87","1,764.49",...,"6,017.27","10,023.44",519.59,783.26,"8,147.21","12,402.50","2,139.69",754.41,"2,449.41","10,166.54"
CY,258.26,245.09,82.40,"15,742.95",206.71,445.22,185.53,111.89,318.24,232.52,...,448.35,322.94,335.06,236.69,194.82,471.13,443.50,277.31,426.74,"1,148.90"
CZ,645.24,294.78,93.33,6.26,"6,184.40","4,495.52",131.61,26.68,428.04,122.40,...,466.47,95.92,235.57,67.09,38.94,841.98,380.97,245.68,437.22,"1,066.65"
DE,484.62,438.93,81.04,8.77,319.80,"7,786.55",147.90,19.17,771.36,130.19,...,478.92,160.94,249.17,146.54,88.12,"1,319.27",638.16,161.11,873.00,"6,716.58"
DK,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
EE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ES,832.86,"3,615.19","8,183.35",41.94,536.91,"7,652.47",347.06,61.94,"19,680.70",330.94,...,"9,483.90","15,261.84",547.87,"1,271.54",341.96,"15,290.39","4,276.91","1,038.27","4,896.72","16,073.53"


## Link Tsec Psec


In [54]:
# ! (matlab code lines 829-836)

# group E_P_T regions together (along rows + columns) and keep sectors disaggregated
E_Psec_Tsec = E_P_T.groupby("sector", axis=0).sum().groupby("sector", axis=1).sum()

# reorder cols/rows and transpose
E_Psec_Tsec = E_Psec_Tsec.loc[sector_labels, sector_labels]
E_Tsec_Psec = E_Psec_Tsec.T

# delete variables that are no longer used to save memory space
del E_Psec_Tsec

# show result
E_Tsec_Psec

sector,Cultivation of paddy rice,Cultivation of wheat,Cultivation of cereal grains nec,"Cultivation of vegetables, fruit, nuts",Cultivation of oil seeds,"Cultivation of sugar cane, sugar beet",Cultivation of plant-based fibers,Cultivation of crops nec,Cattle farming,Pigs farming,...,Landfill of waste: Paper,Landfill of waste: Plastic,Landfill of waste: Inert/metal/hazardous,Landfill of waste: Textiles,Landfill of waste: Wood,Activities of membership organisation n.e.c. (91),"Recreational, cultural and sporting activities (92)",Other service activities (93),Private households with employed persons (95),Extra-territorial organizations and bodies
sector,,,,,,,,,,,,,,,,,,,,,
Cultivation of paddy rice,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of wheat,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of cereal grains nec,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"Cultivation of vegetables, fruit, nuts",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of oil seeds,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Activities of membership organisation n.e.c. (91),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"Recreational, cultural and sporting activities (92)",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Other service activities (93),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


## Link Psec Preg


In [55]:
# ! (matlab code lines 838-840)

# sum the columns of E_P_T and reshape the vector into 163 x 49
E_Psec_Preg = E_P_T.sum(axis=1).unstack(level=0)

# reorder cols/rows
E_Psec_Preg = E_Psec_Preg.loc[sector_labels, region_labels]

# show result
E_Psec_Preg

region,AT,BE,BG,CY,CZ,DE,DK,EE,ES,FI,...,TR,TW,NO,ID,ZA,WA,WL,WE,WF,WM
sector,,,,,,,,,,,,,,,,,,,,,
Cultivation of paddy rice,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of wheat,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of cereal grains nec,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"Cultivation of vegetables, fruit, nuts",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of oil seeds,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Activities of membership organisation n.e.c. (91),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"Recreational, cultural and sporting activities (92)",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Other service activities (93),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


## link FDreg FSsec

$$
\begin{align}
    E_{FS, FDreg} &= diag\biggl(d_{i} \times L_{all, T}\biggr) \times Y_{T, reg}\\
    &+ diag\biggl(d_{i} \times L_{all, T} \times A_{T,O} \times L'_{O,O}\biggr) \times Y_{O, reg}\\
\end{align}
$$


Important notes:

- `*` in **Matlab** = `@` in **Python** (Pandas / Numpy) = **dot product**
- `*` in **Python** = **element-wise multiplication**
- `A * diag(B)` in Matlab = `A @ diag(B)` = `A * B.T`
- `diag(A) * B` in Matlab = `diag(A) @ B` = `( A * B.T ).T`


In [56]:
# ! (matlab code lines 842-854)

# Compute diag(di . Lall-t) . Yrt
E_FS_FDreg_t = (dLt * Y_region.loc[index_t, :].T).T

# reorder cols/rows
E_FS_FDreg_t = E_FS_FDreg_t.loc[index_t, region_labels]

# Compute diag(di . Lt . Ato . Loo)  .  Yro
E_FS_FDreg_o = ((dLt @ AtoLoo) * Y_region.loc[index_o, :].T).T

# reorder cols/rows
E_FS_FDreg_o = E_FS_FDreg_o.loc[index_o, region_labels]

# join both matrices to get the full E_FS_FDreg
E_FS_FDreg = pd.concat([E_FS_FDreg_t, E_FS_FDreg_o], axis=0)

# reorder cols/rows
E_FS_FDreg = E_FS_FDreg.loc[region_and_sector_labels, region_labels]

# group regions together (along rows) and keep sectors disaggregated
E_FSsec_FDreg = E_FS_FDreg.groupby("sector", axis=0).sum()

# reorder cols/rows and transpose
E_FSsec_FDreg = E_FSsec_FDreg.loc[sector_labels, region_labels]
E_FDreg_FSsec = E_FSsec_FDreg.T

# save space by removing variables that are not used anymore
# del E_FS_FDreg_t, E_FS_FDreg_o, E_FSsec_FDreg, E_FS_FDreg

# show result
E_FDreg_FSsec

sector,Cultivation of paddy rice,Cultivation of wheat,Cultivation of cereal grains nec,"Cultivation of vegetables, fruit, nuts",Cultivation of oil seeds,"Cultivation of sugar cane, sugar beet",Cultivation of plant-based fibers,Cultivation of crops nec,Cattle farming,Pigs farming,...,Landfill of waste: Paper,Landfill of waste: Plastic,Landfill of waste: Inert/metal/hazardous,Landfill of waste: Textiles,Landfill of waste: Wood,Activities of membership organisation n.e.c. (91),"Recreational, cultural and sporting activities (92)",Other service activities (93),Private households with employed persons (95),Extra-territorial organizations and bodies
region,,,,,,,,,,,,,,,,,,,,,
AT,1.41,74.65,179.44,932.63,114.64,2.01,14.20,19.91,134.83,135.37,...,10.68,4.45,11.88,10.54,40.82,"1,426.36","1,610.69",768.01,4.51,0.00
BE,46.95,277.46,56.72,"1,476.04",68.56,8.08,89.52,"1,225.74",177.43,147.08,...,30.05,15.30,33.88,14.83,17.38,426.61,"3,763.16","3,451.55",19.22,0.00
BG,4.36,30.69,42.35,254.47,128.62,0.80,9.08,305.41,6.63,4.64,...,11.97,3.05,11.39,2.22,3.01,35.91,"1,097.02",114.51,0.92,0.00
CY,0.82,8.73,6.21,46.23,2.64,0.55,1.73,28.44,2.78,3.44,...,4.54,4.86,4.22,4.79,5.02,73.69,325.92,82.00,1.46,0.00
CZ,5.16,357.23,123.79,601.98,137.64,3.45,29.68,226.31,20.27,84.79,...,108.79,12.57,74.33,22.16,28.34,505.77,"2,056.26",418.79,1.95,0.00
DE,185.75,"1,365.09",630.96,"7,975.66",626.69,49.20,220.36,"4,582.34",407.64,563.40,...,627.80,181.34,834.82,115.47,193.60,"5,996.04","10,561.85","5,112.93",28.87,0.00
DK,3.76,535.20,221.41,383.67,75.60,1.67,8.19,107.50,53.85,73.72,...,10.52,8.06,49.09,5.90,8.02,751.72,"2,283.97",454.95,2.83,0.00
EE,3.11,6.00,21.50,76.63,2.44,0.95,8.02,148.56,16.87,20.71,...,3.90,2.10,2.56,1.11,1.38,26.85,139.29,78.02,1.47,0.00
ES,96.75,"1,114.93",117.18,"5,184.96",64.63,14.42,43.30,"1,541.90",421.73,768.97,...,365.16,98.78,555.52,81.53,115.37,"2,860.10","11,658.55","4,760.67","2,179.89",0.00


## Link Tsec Treg


In [57]:
# ! (matlab code lines 870-872)

# sum the rows of E_P_T and reshape the vector into 163 x 49
E_Tsec_Treg = E_P_T.sum(axis=0).unstack(level=0)

# reorder cols/rows
E_Tsec_Treg = E_Tsec_Treg.loc[sector_labels, region_labels]

# show result
E_Tsec_Treg

region,AT,BE,BG,CY,CZ,DE,DK,EE,ES,FI,...,TR,TW,NO,ID,ZA,WA,WL,WE,WF,WM
sector,,,,,,,,,,,,,,,,,,,,,
Cultivation of paddy rice,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of wheat,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of cereal grains nec,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"Cultivation of vegetables, fruit, nuts",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of oil seeds,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Activities of membership organisation n.e.c. (91),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"Recreational, cultural and sporting activities (92)",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Other service activities (93),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


## link Preg vs FSsec

This section is extra compared to the Nickel analysis

$$
\begin{align}
    E_{P, FS} &= diag\biggl(d_{i}\biggr)  \times L_{all, T}\times diag\biggl(Y_{all, T} \biggr)\\
    &+ diag\biggl(d_{i}\biggr) \times L_{all, T} \times A_{T,O} \times L'_{O,O} \times  diag\biggl(Y_{all, O} \biggr)\\
\end{align}
$$


Important notes:

- `*` in **Matlab** = `@` in **Python** (Pandas / Numpy) = **dot product**
- `*` in **Python** = **element-wise multiplication**
- `A * diag(B)` in Matlab = `A @ diag(B)` = `A * B.T`
- `diag(A) * B` in Matlab = `diag(A) @ B` = `( A * B.T ).T`


In [58]:
# ! (matlab code lines 857-869)

# Compute L . diag(Yt)
diagLt_Yt = L.loc[:, index_t] * Y_global.loc[index_t].T

# Compute diag(di) . L . diag(Yt)
E_P_FS_t = (d_i * diagLt_Yt.T).T

# Compute  (L Ato Loo) . diag(Yo)
LtAtoLoo_diagYo = (L.loc[:, index_t] @ AtoLoo) * Y_global.loc[index_o].T

# Compute diag(di) . (L Ato Loo) . diag(Yo)
E_P_FS_o = (d_i * LtAtoLoo_diagYo.T).T

# join both matrices to get full E_FS_FDreg
E_P_FS = pd.concat([E_P_FS_t, E_P_FS_o], axis=1)

# reorder cols/rows
E_P_FS = E_P_FS.loc[region_and_sector_labels, region_labels]

# group E_P_FS regions together (along rows + columns) and keep sectors disaggregated
E_Psec_FSsec = E_P_FS.groupby("sector", axis=0).sum().groupby("sector", axis=1).sum()

# reorder cols/rows
E_Psec_FSsec = E_Psec_FSsec.loc[sector_labels, sector_labels]

# save space by removing variables that are not used anymore
# del diagLt_Yt, LtAtoLoo_diagYo, E_P_FS_t, E_P_FS_o, E_P_FS

# show result
E_Psec_FSsec

sector,Cultivation of paddy rice,Cultivation of wheat,Cultivation of cereal grains nec,"Cultivation of vegetables, fruit, nuts",Cultivation of oil seeds,"Cultivation of sugar cane, sugar beet",Cultivation of plant-based fibers,Cultivation of crops nec,Cattle farming,Pigs farming,...,Landfill of waste: Paper,Landfill of waste: Plastic,Landfill of waste: Inert/metal/hazardous,Landfill of waste: Textiles,Landfill of waste: Wood,Activities of membership organisation n.e.c. (91),"Recreational, cultural and sporting activities (92)",Other service activities (93),Private households with employed persons (95),Extra-territorial organizations and bodies
sector,,,,,,,,,,,,,,,,,,,,,
Cultivation of paddy rice,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of wheat,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of cereal grains nec,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"Cultivation of vegetables, fruit, nuts",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cultivation of oil seeds,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Activities of membership organisation n.e.c. (91),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"Recreational, cultural and sporting activities (92)",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Other service activities (93),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


# Sankeys preparation


## Preg


In [59]:
# ! (matlab code lines 932-944)

agg_regions = {
    "Australia": region_labels == "AU",
    # "Belgium": region_labels == "BE",
    "Brazil": region_labels == "BR",
    # "Bulgaria": region_labels == "BG",
    # "Canada": region_labels == "CA",
    "China": region_labels == "CN",
    # "Germany": region_labels == "DE",
    # "India": region_labels == "IN",
    "Indonesia": region_labels == "ID",
    "Japan": region_labels == "JP",
    # "Mexico": region_labels == "MX",
    "Netherlands": region_labels == "NL",
    # "Poland": region_labels == "PL",
    # "South Korea": region_labels == "KR",
    # "Russia": region_labels == "RU",
    # "Spain": region_labels == "ES",
    # "USA": region_labels == "US",
    # "RoW Asia Pacific": (region_labels == "WA") | (region_labels == "TW"),
    # "RoW America": region_labels == "WL",
    # "RoW Africa": (region_labels == "WF") | (region_labels == "ZA"),
    # "RoW Middle East": region_labels == "WM",
}
# agg_regions["RoW Europe"] = ~reduce(np.logical_or, list(agg_regions.values()))
agg_regions["other"] = ~reduce(np.logical_or, list(agg_regions.values()))

Agg_Preg = pd.DataFrame(index=region_labels, columns=agg_regions.keys())
for regions in Agg_Preg.columns:
    Agg_Preg.loc[:, regions] = agg_regions[regions].astype(int)

if sum(sum(agg_regions.values())) != len(region_labels):
    print("Warning: some regions are missing in the aggregation matrix")

Agg_Preg

,Australia,Brazil,China,Indonesia,Japan,Netherlands,other
region,,,,,,,
AT,0,0,0,0,0,0,1
BE,0,0,0,0,0,0,1
BG,0,0,0,0,0,0,1
CY,0,0,0,0,0,0,1
CZ,0,0,0,0,0,0,1
DE,0,0,0,0,0,0,1
DK,0,0,0,0,0,0,1
EE,0,0,0,0,0,0,1
ES,0,0,0,0,0,0,1


## FSsec


In [60]:
# ! (matlab code lines 947-963)

agg_sectors = {
    "agrifood": sector_labels[np.r_[0:19, 34:46]],
    "mining": sector_labels[19:34],
    "manufacturing": sector_labels[np.r_[46:85, 92]],
    "machinery_electronics": sector_labels[85:90],
    "auto": sector_labels[90:92],
    "electricity": sector_labels[95:109],
    "gas_water": sector_labels[109:112],
    "construction": sector_labels[112:114],
    "service": sector_labels[np.r_[114:119, 125:138]],
    "transportation": sector_labels[119:125],
    "waste": sector_labels[np.r_[93:95, 138:158]],
    "other_sec": sector_labels[158:],
}

Agg_FSsec = pd.DataFrame(index=sector_labels, columns=agg_sectors.keys())
for sectors in Agg_FSsec.columns:
    Agg_FSsec.loc[:, sectors] = sector_labels.isin(agg_sectors[sectors]).astype(int)

if sum(len(sec) for sec in agg_sectors.values()) != len(sector_labels):
    print("Warning: some sectors are missing in the aggregation matrix")

Agg_FSsec

,agrifood,mining,manufacturing,machinery_electronics,auto,electricity,gas_water,construction,service,transportation,waste,other_sec
sector,,,,,,,,,,,,
Cultivation of paddy rice,1,0,0,0,0,0,0,0,0,0,0,0
Cultivation of wheat,1,0,0,0,0,0,0,0,0,0,0,0
Cultivation of cereal grains nec,1,0,0,0,0,0,0,0,0,0,0,0
"Cultivation of vegetables, fruit, nuts",1,0,0,0,0,0,0,0,0,0,0,0
Cultivation of oil seeds,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
Activities of membership organisation n.e.c. (91),0,0,0,0,0,0,0,0,0,0,0,1
"Recreational, cultural and sporting activities (92)",0,0,0,0,0,0,0,0,0,0,0,1
Other service activities (93),0,0,0,0,0,0,0,0,0,0,0,1


## Psec


In [61]:
# ! (matlab code lines 886-906)

agg_production = {
    "coal": sector_labels[[19]],
    "nickel": sector_labels[[26]],
    "iron": sector_labels[np.r_[24, 71, 72]],
    "copper": sector_labels[np.r_[25, 79, 80]],
    "aluminium": sector_labels[np.r_[27, 75, 76]],
    "precious_metals": sector_labels[np.r_[28, 73, 74]],
}
all_metals = sector_labels[np.r_[23:31, 70:83]]
primary_metals = (
    set(agg_production["nickel"])
    | set(agg_production["iron"])
    | set(agg_production["copper"])
    | set(agg_production["aluminium"])
    | set(agg_production["precious_metals"])
)
agg_production["other_metals"] = all_metals.difference(primary_metals)
agg_production["non_mining"] = sector_labels.difference(
    set(all_metals) | set(agg_production["coal"])
)

Agg_Psec = pd.DataFrame(index=sector_labels, columns=agg_production.keys())
for sectors in Agg_Psec.columns:
    Agg_Psec.loc[:, sectors] = sector_labels.isin(agg_production[sectors]).astype(int)

# Show
Agg_Psec

,coal,nickel,iron,copper,aluminium,precious_metals,other_metals,non_mining
sector,,,,,,,,
Cultivation of paddy rice,0,0,0,0,0,0,0,1
Cultivation of wheat,0,0,0,0,0,0,0,1
Cultivation of cereal grains nec,0,0,0,0,0,0,0,1
"Cultivation of vegetables, fruit, nuts",0,0,0,0,0,0,0,1
Cultivation of oil seeds,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...
Activities of membership organisation n.e.c. (91),0,0,0,0,0,0,0,1
"Recreational, cultural and sporting activities (92)",0,0,0,0,0,0,0,1
Other service activities (93),0,0,0,0,0,0,0,1


## Tsec


In [62]:
# ! (matlab code lines 908-914)

# Remove the "coal" and "non_mining" columns to get Agg_Tsec
Agg_Tsec = Agg_Psec[["nickel", "iron", "copper", "aluminium", "precious_metals", "other_metals"]]
Agg_Tsec

,nickel,iron,copper,aluminium,precious_metals,other_metals
sector,,,,,,
Cultivation of paddy rice,0,0,0,0,0,0
Cultivation of wheat,0,0,0,0,0,0
Cultivation of cereal grains nec,0,0,0,0,0,0
"Cultivation of vegetables, fruit, nuts",0,0,0,0,0,0
Cultivation of oil seeds,0,0,0,0,0,0
...,...,...,...,...,...,...
Activities of membership organisation n.e.c. (91),0,0,0,0,0,0
"Recreational, cultural and sporting activities (92)",0,0,0,0,0,0
Other service activities (93),0,0,0,0,0,0


## Sankey #1


In [63]:
# # ! (matlab code lines 965-982)

# TOT = E_Preg_Treg.sum().sum()

# # shares_Sankey_Tsec_Psec = (Agg_Psec.T @ E_Tsec_Psec.T @ Agg_Tsec) # / TOT
# # shares_Sankey_Psec_Preg = (Agg_Psec.T @ E_Psec_Preg @ Agg_Preg) # / TOT
# shares_Sankey_Preg_Treg = Agg_Preg.T @ E_Preg_Treg @ Agg_Preg # / TOT
# shares_Sankey_Treg_FDreg = Agg_Preg.T @ E_Treg_FDreg @ Agg_Preg # / TOT
# shares_Sankey_FDreg_FSsec = Agg_Preg.T @ E_FDreg_FSsec @ Agg_FSsec # / TOT

# perspectives = [
#     shares_Sankey_Preg_Treg.sum(axis=1),
#     shares_Sankey_Treg_FDreg.sum(axis=1),
#     shares_Sankey_Treg_FDreg.sum().T,
#     shares_Sankey_FDreg_FSsec.sum().T,
# ]
# linkages = [dd
#     shares_Sankey_Preg_Treg.stack(),
#     shares_Sankey_Treg_FDreg.stack(),
#     shares_Sankey_FDreg_FSsec.stack(),
# ]

# # Show results
# threshold = 0.001
# print(stress, "\n")
# for i in range(3):
#     print(f"linkage {i+1}")
#     mask = linkages[i] > threshold
#     print(linkages[i][mask], "\n\n")
# for i in range(4):
#     print(f"perspectives {i+1}")
#     mask = perspectives[i] > threshold
#     print(perspectives[i][mask], "\n\n")

## Sankey #2


In [67]:
# ! (matlab code lines 985-995)

shares_Sankey_FSsec_Psec = Agg_Preg.T @ E_Psec_Preg.T @ Agg_Psec  # / E_Psec_Preg.sum().sum()
shares_Sankey_Psec_Tsec = Agg_Psec.T @ E_Tsec_Psec.T @ Agg_Tsec  # / E_Psec_Preg.sum().sum()
shares_Sankey_Tsec_Treg = Agg_Tsec.T @ E_Tsec_Treg @ Agg_Preg  # / E_Psec_Preg.sum().sum()

perspectives = [
    shares_Sankey_FSsec_Psec.sum(axis=1),
    shares_Sankey_Psec_Tsec.sum(axis=1),
    shares_Sankey_Psec_Tsec.sum().T,
    shares_Sankey_Tsec_Treg.sum().T,
]
linkages = [
    shares_Sankey_FSsec_Psec.stack(),
    shares_Sankey_Psec_Tsec.stack(),
    shares_Sankey_Tsec_Treg.stack(),
]

# Show results
threshold = 0.001
print(stress, "\n")
for i in range(3):
    print(f"linkage {i+1}")
    mask = linkages[i] > threshold
    print(linkages[i][mask], "\n\n")
for i in range(4):
    print(f"perspectives {i+1}")
    mask = perspectives[i] > threshold
    print(perspectives[i][mask], "\n\n")

biodiv 

linkage 1
Australia  coal               2,597,990.70
           nickel               697,138.44
           iron               8,902,966.09
           copper             1,176,498.59
           aluminium          7,162,795.88
           precious_metals    2,008,497.40
           other_metals       1,625,382.30
           non_mining           221,834.93
Brazil     coal                 116,995.35
           nickel               163,756.57
           iron               3,067,030.74
           copper               266,851.56
           aluminium          2,437,525.79
           precious_metals      398,737.39
           other_metals         173,784.72
           non_mining            22,993.00
China      coal                 709,105.92
           nickel                32,998.36
           iron               2,292,164.90
           copper               268,666.91
           aluminium            678,352.41
           precious_metals      425,101.01
           other_metals         535

In [65]:
# Save results
metal = "metals"
year = 2014
filename = f"{metal}_{stress}_{year}_sankey_2"
mapping = {"level_0": "source_country", "level_1": "target_country", 0: "value"}
stages = ["mining", "processing", "consumption", "End-use"]

for i, _ in enumerate(linkages):
    linkages[i] = linkages[i].reset_index().rename(columns=mapping)
    linkages[i]["source_stage"] = stages[i]
    linkages[i]["target_stage"] = stages[i + 1]
    linkages[i] = linkages[i][
        ["source_stage", "source_country", "target_stage", "target_country", "value"]
    ]

pd.concat(linkages, axis=0).to_csv(f"./results/{metal}/{filename}.csv", index=False)

### Biodiversity

![S24 original](./img/metals/S24.png)
![S24 reproduced](./img/metals/S24_reproduced.png)


### Land use

![Land use](./img/metals/metals_landuse.png)

### Greenhouse gases

![ghg](./img/metals/metals_ghg.png)

### Blue water consumption

![water](./img/metals/metals_water.png)


## Sankey #3


In [66]:
# # ! (matlab code lines 998-1009)

# shares_Sankey_FSsec_Psec = Agg_FSsec.T @ E_Psec_FSsec.T @ Agg_Psec / E_Psec_FSsec.sum().sum()
# # shares_Sankey_Preg_Psec = (Agg_Preg.T @ E_Psec_Preg.T @ Agg_Psec) #/ TOT
# shares_Sankey_Psec_Tsec = Agg_Psec.T @ E_Tsec_Psec.T @ Agg_Tsec / E_Tsec_Psec.sum().sum()
# shares_Sankey_Tsec_Treg = Agg_Tsec.T @ E_Tsec_Treg @ Agg_Preg / E_Tsec_Treg.sum().sum()

# perspectives = [
#     shares_Sankey_FSsec_Psec.sum(axis=1),
#     shares_Sankey_Psec_Tsec.sum(axis=1),
#     shares_Sankey_Psec_Tsec.sum().T,
#     shares_Sankey_Tsec_Treg.sum().T,
# ]
# linkages = [
#     shares_Sankey_FSsec_Psec.stack(),
#     shares_Sankey_Psec_Tsec.stack(),
#     shares_Sankey_Tsec_Treg.stack(),
# ]

# # Show results
# threshold = 0.001
# print(stress, "\n")
# for i in range(3):
#     print(f"linkage {i+1}")
#     mask = linkages[i] > threshold
#     print(linkages[i][mask], "\n\n")
# for i in range(4):
#     print(f"perspectives {i+1}")
#     mask = perspectives[i] > threshold
#     print(perspectives[i][mask], "\n\n")